# MNIST

In [1]:
%load_ext autoreload
%autoreload 2
# Import dependencies
import os
import importlib
import numpy as np
import matplotlib.pyplot as plt
import time
import torchvision
import seaborn as sns
import numpy as np
import pytorch_lightning as pl
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision.utils import save_image
import torch
from tqdm import tqdm
from datetime import datetime
import yaml
import h5py
from copy import deepcopy
import ambiguous.models.cvae
from ambiguous.models.cvae import *
from ambiguous.dataset.dataset import partition_dataset
import wandb
from pytorch_lightning.loggers import WandbLogger

In [2]:
TRAIN_CVAE = True
LOCAL_CKPT = False

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Seed for reproducibility
torch.manual_seed(42)
print(device)
root='/home/mila/n/nizar.islah/expectation-clamp/'

cpu


In [3]:
transform = transforms.Compose([
    transforms.ToTensor()
])
dataset = datasets.MNIST(root=root, download=True, train=True, transform=transform)
train_set, val_set = torch.utils.data.random_split(dataset, [round(0.8*len(dataset)), round(0.2*len(dataset))])
test_set = datasets.MNIST(root=root, download=True, train=False, transform=transform)
# Dataloaders
batch_size = 64
train_loader = DataLoader(train_set, batch_size=batch_size, num_workers=2, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, num_workers=2)
test_loader = DataLoader(test_set, batch_size=batch_size, num_workers=2)

In [4]:
s = range(100)
t = tqdm(total=len(s))
for x in s:
    t.update()
t.refresh()  # force print final state

t.reset()  # reuse bar
for x in s:
    t.update()
t.close()  # close the bar permanently

100%|██████████| 100/100 [00:00<00:00, 159783.01it/s]


In [6]:
x,t=next(iter(train_loader))
torchvision.utils.save_image(x, 'image.png', nrow=8)

In [7]:
n_cls=10
img_size=28
lr=1e-3
num_epochs=50
latent_dim = 10

onehot = torch.zeros(n_cls, n_cls).to(device)
onehot = onehot.scatter_(1, torch.LongTensor(range(n_cls)).view(n_cls,1).to(device), 1).view(n_cls, n_cls, 1, 1)
fill = torch.zeros([n_cls, n_cls, img_size, img_size]).to(device)
for i in range(n_cls):
    fill[i, i, :, :] = 1

if TRAIN_CVAE:
    model = Conv_CVAE(
    latent_dim = latent_dim,
        n_cls=n_cls
    ).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss(reduction='sum')
    n_train = len(train_loader)
    n_val = len(val_loader)
    for i in tqdm(range(num_epochs)):
        running_loss = 0
        for idx, (images, labels) in tqdm(enumerate(train_loader)):
            images = images.to(device)
            labels = labels.to(device)
            labels_fill_ = fill[labels]
            y_ = (torch.rand(images.size(0), 1) * n_cls).type(torch.LongTensor).squeeze().to(device)
            y_label_ = onehot[y_]
            rec, mu, logvar = model((images, labels_fill_, y_label_))
            loss_dict = model.loss_function(rec, images, mu, logvar)
            running_loss += loss_dict['loss'].item()/batch_size
            optimizer.zero_grad()
            loss_dict['loss'].backward()
            optimizer.step()
            # wandb.log(loss_dict)
        val_loss = 0
        for _, (images, labels) in tqdm(enumerate(val_loader)):
            with torch.no_grad():
                images = images.to(device)
                labels = labels.to(device)
                labels_fill_ = fill[labels]
                y_ = (torch.rand(images.size(0), 1) * n_cls).type(torch.LongTensor).squeeze()
                y_label_ = onehot[y_]
                rec, mu, logvar = model((images, labels_fill_, y_label_))
                torchvision.utils.save_image(rec, "reconstruction_valid.pdf")
                loss_dict = model.loss_function(rec, images, mu, logvar)
                val_loss += loss_dict['loss'].item()/batch_size
                loss_dict = {'loss_val':loss_dict['loss'], 'kld_val':loss_dict['kld'], 'rec_val':loss_dict['rec']}
                # wandb.log(loss_dict)
        torch.save(model, 'conv_cvae_mnist.pth')
        print(f"Epoch: {i+1} \t Train Loss: {running_loss:.2f} \t Val Loss: {val_loss:.2f}")
    
else:
    if LOCAL_CKPT:
        ckpt_path = ""
        ckpt = torch.load(ckpt_path)
    else:
        # run = wandb.init()
        # artifact = run.use_artifact('team-nizar/EMNIST_CVAE/model-pkia9rhl:v29', type='model')
        # artifact_dir = artifact.download()
        ckpt_path = ""
        ckpt = torch.load(ckpt_path)
        model = EMNIST_CVAE(latent_dim, enc_layers, dec_layers, n_classes=26, conditional=True).to(device)
        model.load_state_dict(ckpt['state_dict'])
    
model.eval()

10 16


7it [00:43,  6.19s/it][00:00<?, ?it/s]
  0%|          | 0/50 [00:44<?, ?it/s]


KeyboardInterrupt: 

In [7]:
onehot

tensor([[[[1.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]]],


        [[[0.]],

         [[1.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]]],


        [[[0.]],

         [[0.]],

         [[1.]],

      

In [8]:
model.eval()
for _, (images, labels) in enumerate(val_loader):
    with torch.no_grad():
        images = images.to(device)
        labels = labels.to(device)
        labels_fill_ = fill[labels]
        y_ = (torch.rand(images.size(0), 1) * n_cls).type(torch.LongTensor).squeeze()
        y_label_ = onehot[y_]
        rec, mu, logvar = model((images, labels_fill_, y_label_))
    break

torchvision.utils.save_image(rec, "example.pdf")

In [14]:
torch.cuda.empty_cache()

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [11]:
y_label_

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [10]:
for _, (images, labels) in enumerate(val_loader):
    with torch.no_grad():
        images = images.to(device)
        labels = labels.to(device)
        labels_fill_ = fill[labels]
        y_ = (torch.rand(images.size(0), 2) * n_cls).type(torch.LongTensor).squeeze()
        y_label_ = onehot[y_]
        rec, mu, logvar = model((images, labels_fill_, y_label_))
    break
    
torchvision.utils.save_image(rec, "example_ambi.pdf")

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.